# Please remind that this code is not the final code of the paper. The final code will be released upon acceptance. The crrent code differs with the main code in some parts for hyperparameter tuning and evaluation.

### This part is used to prepare the input data to the model

In [ ]:
import pandas as pd
import numpy as np

df_alt = pd.read_excel("altimetry.xlsx")   # columns: DateTime, Predicted_WSE, Sensor
df_in_situ = pd.read_excel("in_situ.xlsx")  # columns: DateTime, In_Situ_WSE

# Convert DateTime to datetime format
df_alt['DateTime'] = pd.to_datetime(df_alt['DateTime'])
df_in_situ['DateTime'] = pd.to_datetime(df_in_situ['DateTime'])

# Sanity check: make sure datetimes match
assert all(df_alt['DateTime'] == df_in_situ['DateTime']), "DateTimes do not match!"

df_alt['Year'] = df_alt['DateTime'].dt.year
df_alt['DOY'] = df_alt['DateTime'].dt.dayofyear

df_alt['DOY_sin'] = np.sin(2 * np.pi * df_alt['DOY'] / 365)
df_alt['DOY_cos'] = np.cos(2 * np.pi * df_alt['DOY'] / 365)

def map_sensor(sensor_name):
    if sensor_name.startswith("S3A"):
        return 0
    elif sensor_name.startswith("S3B"):
        return 1
    elif sensor_name.startswith("S6"):
        return 2
    elif sensor_name.startswith("SWOT"):
        return 3
    else:
        return -1  # Unknown sensor

df_alt['Sensor_Code'] = df_alt['Sensor'].apply(map_sensor)

# Check for any -1 values in Sensor_Code
unknown_sensors = df_alt[df_alt['Sensor_Code'] == -1]

# Display result
if not unknown_sensors.empty:
    print("Unrecognized sensors found:")
    print(unknown_sensors[['DateTime', 'Sensor']])
else:
    print("All sensor names recognized and mapped successfully.")

df_alt['In_Situ_WSE'] = df_in_situ['In_Situ_WSE']

df_alt

In [ ]:
# Read Excel files on PC office
df_weather = pd.read_excel("ERA5.xlsx")   # columns: DateTime, Predicted_WSE, Sensor

df_weather['DATE'] = pd.to_datetime(df_weather['DATE'])
df_weather = df_weather.sort_values('DATE').reset_index(drop=True)
df_weather = df_weather.set_index('DATE')
df_weather

n = 30 # length of timeseries

temp_series = []
prec_series = []
evap_series = []
valid_flags = []  # to track which rows have full history

for timestamp in df_alt['DateTime']:
    start_date = timestamp.normalize() - pd.Timedelta(days=n-1)
    end_date = timestamp.normalize()

    # Slice n days up to and including the day before the timestamp
    ts_slice = df_weather.loc[start_date:end_date]

    if len(ts_slice) == n:
        temp_series.append(ts_slice['Temp_Celsius'].values.tolist())
        prec_series.append(ts_slice['Prec'].values.tolist())
        evap_series.append(ts_slice['Evap'].values.tolist())
        valid_flags.append(True)

    else:
        temp_series.append(None)
        prec_series.append(None)
        evap_series.append(None)
        valid_flags.append(False)

# Add to df_pred
df_alt['Temp_series'] = temp_series
df_alt['Prec_series'] = prec_series
df_alt['Evap_series'] = evap_series
df_alt['Valid'] = valid_flags

df_alt_valid = df_alt[df_alt['Valid'] == True].reset_index(drop=True) # remove data that had not complete timeseries

In [ ]:
df_alt_valid.to_excel(f"alt_ERA5_env_{n}days.xlsx", index=False)

### Read the prepared data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast

df_data = pd.read_excel(f"alt_ERA5_env_{n}days.xlsx")   # columns: DateTime, Predicted_WSE, Sensor

input_point_features = np.column_stack([
    df_data['Predicted_WSE'].values,
    df_data['DOY_sin'].values,
    df_data['DOY_cos'].values,
    df_data['Sensor_Code'].values
])

input_T = np.column_stack([
    np.array([ast.literal_eval(s) for s in df_data['Temp_series'].values])
])

input_P = np.column_stack([
    np.array([ast.literal_eval(s) for s in df_data['Prec_series'].values])
])
input_Ev = np.column_stack([
    np.array([ast.literal_eval(s) for s in df_data['Evap_series'].values])
])
target = np.column_stack([
    df_data['In_Situ_WSE'].values
])

### Altimetry vs In_situ

In [ ]:
plt.plot(input_point_features[:, 0], 'b') #blue is altimetry
plt.plot(target[:, 0], 'r') # red is in-situ



rmse_value = np.sqrt(np.mean((input_point_features[:, 0] - target[:, 0])**2))
pcc_value = np.corrcoef(target[:, 0], input_point_features[:, 0])[0, 1] * 100  # Pearson Correlation Coefficient
print("########################################")
print("RMSE:", rmse_value)
print("PCC :", pcc_value)

### Preprocessing

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler


def log_scale_transform_timeseries(X):
    
    shift_value_X = np.abs(np.min(X)) + 1  # Shift X to ensure positive values
    
    X_shifted = X + shift_value_X
    
    X_log = np.log(X_shifted)

    transformerX = RobustScaler().fit(X_log)
    X_trans = transformerX.transform(X_log)
    min_max_scalerX = MinMaxScaler().fit(X_trans)
    X_trans2 = min_max_scalerX.transform(X_trans)

    return X_trans2, transformerX, min_max_scalerX, shift_value_X

input_T_flat = input_T.reshape(-1, 1) # stacks each row of input_T in a single column
input_T_flat_rescaled, transformerT, min_max_scalerT, shift_value_T  = log_scale_transform_timeseries(input_T_flat)
input_T_rescaled = input_T_flat_rescaled.reshape(-1, input_T.shape[1])

input_P_flat = input_P.reshape(-1, 1) # stacks each row of input_T in a single column
input_P_flat_rescaled, transformerP, min_max_scalerP, shift_value_P  = log_scale_transform_timeseries(input_P_flat)
input_P_rescaled = input_P_flat_rescaled.reshape(-1, input_P.shape[1])

input_Ev_flat = input_Ev.reshape(-1, 1) # stacks each row of input_T in a single column
input_Ev_flat_rescaled, transformerEv, min_max_scalerEv, shift_value_Ev  = log_scale_transform_timeseries(input_Ev_flat)
input_Ev_rescaled = input_Ev_flat_rescaled.reshape(-1, input_Ev.shape[1])


def log_scale_transform(X, Y):
    
    shift_value_X = np.abs(np.min(X)) + 1  # Shift X to ensure positive values
    shift_value_Y = np.abs(np.min(Y)) + 1  # Shift Y similarly
    
    X_shifted = X + shift_value_X
    Y_shifted = Y + shift_value_Y
    
    X_log = np.log(X_shifted)
    Y_log = np.log(Y_shifted)

    transformerX = RobustScaler().fit(X_log)
    X_trans = transformerX.transform(X_log)
    min_max_scalerX = MinMaxScaler().fit(X_trans)
    X_trans2 = min_max_scalerX.transform(X_trans)

    transformerY = RobustScaler().fit(np.reshape(Y_log,(-1,1)))
    Y_trans = transformerY.transform(np.reshape(Y_log,(-1,1)))
    min_max_scalerY = MinMaxScaler().fit(Y_trans)
    Y_trans2 = min_max_scalerY.transform(Y_trans)

    return X_trans2, Y_trans2, transformerX, transformerY, min_max_scalerX, min_max_scalerY, shift_value_X, shift_value_Y

input_point_rescaled, y_rescaled, transformerPoint, transformerY, min_max_scalerPoint, min_max_scalerY, shift_value_Point, shift_value_Y  = log_scale_transform(input_point_features, target)

def y_to_initial_scale(y, min_max_scaler, transformer, shift):
    y = min_max_scaler.inverse_transform(y.reshape(-1, 1))
    y = transformer.inverse_transform(y)
    return np.exp(y) - shift - 1

### EILSTMNet

In [ ]:
import torch
import torch.nn as nn

class CustomLSTMstack(nn.Module):

    def __init__(self, input_size, hidden_sizes, proj_size = None, dropout_rate = 0):

        # input_size is the number of input features
        # hidden_sizes must be a list like [6, 3, 2]
        # proj_size only is considered for the last layer 
        # dropout_rate only is not considered for the last layer

        super().__init__()

        self.layers = nn.ModuleList()
        self.hidden_sizes = hidden_sizes

        for i in range(len(hidden_sizes)):
            in_size = input_size if i==0 else hidden_sizes[i-1]
            hidden_size = hidden_sizes[i]
            is_last = ( i == len(hidden_sizes) - 1) # True or False

            lstm = nn.LSTM(
                
                input_size = in_size,
                hidden_size = hidden_size,
                num_layers = 1,
                proj_size = proj_size if is_last and proj_size is not None else 0,
                dropout = dropout_rate if not is_last and len(hidden_sizes) > 1 else 0,
                bias = True,
                bidirectional=False,
                batch_first = True,

            )

            self.layers.append(lstm)

    def forward(self, x):

        for i, lstm in enumerate(self.layers):
            
            x, (hn, cn) = lstm(x) # so in the stacked format, the outputs are passed to the next lstm not the hidden stated

        return hn[-1] #shape: (samples, proj_size)

class LSTMRegressor(nn.Module):

    def __init__(self, seq_input_size, lstm_hidden_sizes, proj_size,
                 point_features_shape, mlp_layer_sizes, target_size, dropout_rate = 0):
        super().__init__()

        # seq_input_size is the number of input features for LSTM
        # lstm_hidden_sizes must be a list like [6, 3, 2] for LSTM
        # proj_size only is considered for the last layer for LSTM
        # dropout_rate only is not considered for the last layer for LSTM and also works for MLP
        # point_features_shape is a number of point-based features
        # mlp_layer_sizes is a list like [8, 10, 12]
        # target_size is a simple number like 1

        self.lstm_stack = CustomLSTMstack(

            input_size = seq_input_size,
            hidden_sizes = lstm_hidden_sizes,
            proj_size = proj_size,
            dropout_rate = dropout_rate

        )

        lstm_output_size = proj_size if proj_size is not None else lstm_hidden_sizes[-1]
        mlp_input_size = lstm_output_size + point_features_shape

        mlp_layers = []
        mlp_layer_sizes = [mlp_input_size] + mlp_layer_sizes

        for i in range(len(mlp_layer_sizes) - 1):
            mlp_layers.append(nn.Linear(mlp_layer_sizes[i], mlp_layer_sizes[i+1]))
            mlp_layers.append(nn.ReLU())
            mlp_layers.append(nn.Dropout(dropout_rate))

        mlp_layers.append(nn.Linear(mlp_layer_sizes[-1], target_size))

        self.mlp = nn.Sequential(*mlp_layers)

    def forward(self,x_seq, x_point):
        
        lstm_lasthidden = self.lstm_stack(x_seq)
        combined = torch.cat([lstm_lasthidden, x_point], dim = 1)

        return self.mlp(combined)

### Hyperparameter tuning

#### The fine/tuning method reported here is not the same as what is reported in the paper. The exact code of the article will be released upon acceptance of the paper

In [ ]:
import pandas as pd
import os

def write_hyper_results(params, avg_metrics):


    results_csv = f"alt_env_{n}days_hyperparam_results.csv"

    # Convert and write immediately
    row = {**params, **avg_metrics}
    df = pd.DataFrame([row])

    # Append to file
    if not os.path.exists(results_csv):
        df.to_csv(results_csv, index=False)
    else:
        df.to_csv(results_csv, mode='a', index=False, header=False)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from torch.utils.data import DataLoader, TensorDataset, Subset
import torch.optim as optim
from itertools import product
from sklearn.model_selection import KFold

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_seq = np.stack([input_P_rescaled, input_Ev_rescaled, input_T_rescaled], axis=2)
x_seq_tensor = torch.tensor(x_seq, dtype = torch.float32).to(device)
x_point_tensor = torch.tensor(input_point_rescaled, dtype = torch.float32).to(device)
y_tensor = torch.tensor(y_rescaled, dtype=torch.float32).to(device)

dataset = TensorDataset(x_seq_tensor, x_point_tensor, y_tensor)

# Define search space
param_grid = {
    'lr': [0.0005, 0.0001, 0.005, 0.001, 0.05, 0.01],
    'epochs': [100, 200, 300],
    'batch_size': [256, 128, 64, 32, 16, 8],
    'lstm_hidden_sizes': [[64, 32, 16, 8, 4]],
    'mlp_layer_sizes': [[64, 32, 16, 8, 4]],
    'dropout_rate': [0, 0.1, 0.2],
}

# Generate all combinations of hyperparameters
param_combinations = list(product(*param_grid.values()))
param_names = list(param_grid.keys())

# Track best
best_params = None
best_score = float('inf')

# Cross-validation
kf = KFold(n_splits=3, shuffle=True, random_state=42)

param_grid_scores = []

total_combinations = len(param_combinations)
total_combinations_counter = 1

for param_values in param_combinations:
    params = dict(zip(param_names, param_values))
    # print(f"\nTrying params: {params}")
    print(f"{total_combinations_counter} out of {total_combinations}")
    total_combinations_counter = total_combinations_counter + 1

    fold_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=params['batch_size'], shuffle=True)
        # print("train_loader", train_loader)
        val_loader = DataLoader(val_subset, batch_size=params['batch_size'])
        # print("val_loader", val_loader)

        model = LSTMRegressor(
            seq_input_size=x_seq_tensor.shape[-1],
            lstm_hidden_sizes=params['lstm_hidden_sizes'],
            proj_size=None,
            point_features_shape=x_point_tensor.shape[-1],
            mlp_layer_sizes=params['mlp_layer_sizes'],
            target_size=y_tensor.shape[-1],
            dropout_rate=params['dropout_rate']
        ).to(device)

        criterion = nn.MSELoss()
        # optimizer = optim.Adam(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
        optimizer = optim.Adam(model.parameters(), lr=params['lr'])

        # Training loop
        for epoch in range(params['epochs']):
            model.train()
            for x_seq_batch, x_point_batch, y_batch in train_loader:
                # print("x_seq_batch", x_seq_batch.shape)
                # print("x_point_batch", x_point_batch.shape)
                # print("y_batch", y_batch.shape)
                x_seq_batch = x_seq_batch.to(device)
                x_point_batch = x_point_batch.to(device)
                y_batch = y_batch.to(device)

                preds = model(x_seq_batch, x_point_batch)
                loss = criterion(preds, y_batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        # Validation
        model.eval()
        val_loss = 0.0
        count = 0
        with torch.no_grad():

            metrics = {'RMSE': [], 'MAPE': [], 'R2': [], 'PCC': []}

            all_y_true = []
            all_y_pred = []

            for x_seq_batch, x_point_batch, y_batch in val_loader:

                x_seq_batch = x_seq_batch.to(device)
                x_point_batch = x_point_batch.to(device)
                y_batch = y_batch.to(device)

                preds = model(x_seq_batch, x_point_batch)

                all_y_true.append(y_batch.detach().cpu())
                all_y_pred.append(preds.detach().cpu())


            # ⬇️ Concatenate all batches into one tensor for the fold
            y_true_full = torch.cat(all_y_true, dim=0).numpy()
            y_pred_full = torch.cat(all_y_pred, dim=0).numpy()

            # print("y_true", y_true_full.shape)
            # print("y_pred", y_true_full.shape)

            # ⬇️ Inverse transform back to real-world scale
            y_true_real = y_to_initial_scale(y_true_full[:, 0], min_max_scalerY, transformerY, shift_value_Y).flatten()
            y_pred_real = y_to_initial_scale(y_pred_full[:, 0], min_max_scalerY, transformerY, shift_value_Y).flatten()


            # Metrics
            # ✅ Compute fold-level metrics
            rmse = np.sqrt(np.mean((y_pred_real - y_true_real) ** 2))
            pcc = np.corrcoef(y_true_real, y_pred_real)[0, 1] * 100

            # ✅ Append metrics for this fold
            metrics['RMSE'].append(rmse)
            metrics['PCC'].append(pcc)

        fold_scores.append(metrics)

    avg_metrics = {
    metric: np.mean([fold[metric][0] for fold in fold_scores])  # unwrap [value]
    for metric in ['RMSE', 'PCC']
    }

    # param_grid_scores.append((params, avg_metrics))
    write_hyper_results(params, avg_metrics)

### Using the best model

#### The validation method reported here is not the same as what is reported in the paper. The exact code of the article will be released upon acceptance of the paper

In [ ]:
import pandas as pd
import ast
from torch.utils.data import random_split

df = pd.read_csv(f'alt_env_{n}days_hyperparam_results.csv')
df_sorted = df.sort_values(by="RMSE", ascending=True)
best_params = df_sorted.iloc[0]

best_params
lr = float(best_params['lr'])
epochs = int(best_params['epochs'])
batch_size = int(best_params['batch_size'])
lstm_hidden_sizes = ast.literal_eval(best_params['lstm_hidden_sizes'])
mlp_layer_sizes = ast.literal_eval(best_params['mlp_layer_sizes'])
dropout_rate = float(best_params['dropout_rate'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_seq = np.stack([input_P_rescaled, input_Ev_rescaled, input_T_rescaled], axis=2)
x_seq_tensor = torch.tensor(x_seq, dtype = torch.float32).to(device)
x_point_tensor = torch.tensor(input_point_rescaled, dtype = torch.float32).to(device)
y_tensor = torch.tensor(y_rescaled, dtype=torch.float32).to(device)

dataset = TensorDataset(x_seq_tensor, x_point_tensor, y_tensor)

train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size)

model = LSTMRegressor(
    seq_input_size=x_seq_tensor.shape[-1],
    lstm_hidden_sizes=lstm_hidden_sizes,
    proj_size=None,
    point_features_shape=x_point_tensor.shape[-1],
    mlp_layer_sizes=mlp_layer_sizes,
    target_size=y_tensor.shape[-1],
    dropout_rate=dropout_rate
).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

# Training loop
for epoch in range(epochs):
    model.train()
    for x_seq_batch, x_point_batch, y_batch in train_loader:

        x_seq_batch = x_seq_batch.to(device)
        x_point_batch = x_point_batch.to(device)
        y_batch = y_batch.to(device)

        preds = model(x_seq_batch, x_point_batch)
        loss = criterion(preds, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

model.eval()
val_loss = 0.0
count = 0
with torch.no_grad():

    for x_seq_batch, x_point_batch, y_batch in test_loader:

        x_seq_batch = x_seq_batch.to(device)
        x_point_batch = x_point_batch.to(device)
        y_batch = y_batch.to(device)

        preds = model(x_seq_batch, x_point_batch)

        all_y_true.append(y_batch.detach().cpu())
        all_y_pred.append(preds.detach().cpu())


    # ⬇️ Concatenate all batches into one tensor for the fold
    y_true_full = torch.cat(all_y_true, dim=0).numpy()
    y_pred_full = torch.cat(all_y_pred, dim=0).numpy()

    # ⬇️ Inverse transform back to real-world scale
    y_true_real = y_to_initial_scale(y_true_full[:, 0], min_max_scalerY, transformerY, shift_value_Y).flatten()
    y_pred_real = y_to_initial_scale(y_pred_full[:, 0], min_max_scalerY, transformerY, shift_value_Y).flatten()

    # Metrics
    # ✅ Compute fold-level metrics
    rmse = np.sqrt(np.mean((y_pred_real - y_true_real) ** 2))
    pcc = np.corrcoef(y_true_real, y_pred_real)[0, 1] * 100

    print('RMSE', rmse)